In [1]:
# AutoEncoders
import tensorflow as tf


class AutoEncoder:
    def __init__(self, input_dim, hidden_dim, learning_rate=0.01, epochs=250):
        self.epochs = epochs
        self.learning_rate = learning_rate
        
        x = tf.placeholder(tf.float32, shape=[None, input_dim])
        
        with tf.name_scope('encode'):
            weights = tf.random.normal([input_dim, hidden_dim], dtype=tf.float32, name="weights")
            biases = tf.Variable(tf.zeros([hidden_dim], dtype=tf.float32, name="biases"))
            encoded = tf.nn.tanh(tf.add(tf.matmul(x, weights), biases))
            
        with tf.name_scope('decode'):
            weights = tf.random.normal([hidden_dim, input_dim], dtype=tf.float32, name="weights")
            biases = tf.Variable(tf.zeros([input_dim], dtype=tf.float32, name="biases"))
            decoded = tf.matmul(encoded, weights) + biases
            
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
        
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, decoded))))
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for ep in range(self.epochs):
                for i in range(num_samples):
                    loss, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: [data[i]]})
                if self.epochs % 10 == 0:
                    print("epoch : {0}, loss: {1}".format(ep, loss))
            self.saver.save(sess, '/tmp/autoencoder.ckpt')
    
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, '/tmp/autoencoder.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
        print('input', data)
        print('compresses', hidden)
        print('output', reconstructed)
        return reconstructed


In [ ]:
from sklearn import datasets

hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])

ae = AutoEncoder(input_dim, hidden_dim)
ae.train(data)

ae.test([[2, 4, 6, 8]])




